In [ ]:
# default_exp loss
%load_ext lab_black # nb_black if running in jupyter
%load_ext autoreload # automatically reload python modules if there are changes in the 
%autoreload 2

# Loss

> In this notebook evaluation metrics are defined and the ML models are evaluated on the real data

In [ ]:
#hide
from nbdev.showdoc import *